In [ ]:
%pip install langchain langchain_openai langchain-community --upgrade

In [3]:
import os
from uuid import uuid4

unique_id = uuid4().hex[0:8]
os.environ["LANGCHAIN_PROJECT"] = f"Prompt Tracing Walkthrough - {unique_id}"
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"

# Retrieve API keys with validation
langsmith_api_key = os.getenv("LANGSMITH_API_KEY")
openai_api_key = os.getenv("OPENAI_API_KEY")
serpapi_api_key = os.getenv("SERPAPI_API_KEY")

if not langsmith_api_key:
    raise ValueError("❌ LANGSMITH_API_KEY is not set!")
if not openai_api_key:
    raise ValueError("❌ OPENAI_API_KEY is not set!")
if not serpapi_api_key:
    raise ValueError("❌ SERPAPI_API_KEY is not set!")

os.environ["LANGSMITH_API_KEY"] = langsmith_api_key
os.environ["OPENAI_API_KEY"] = openai_api_key
os.environ["SERPAPI_API_KEY"] = serpapi_api_key

print("✅ All API keys are set successfully.")
print(f"LANGCHAIN_PROJECT: {os.environ['LANGCHAIN_PROJECT']}")

✅ All API keys are set successfully.
LANGCHAIN_PROJECT: Prompt Tracing Walkthrough - a779fcec


In [10]:
from langsmith import Client

langsmith_client = Client(api_key=langsmith_api_key)

In [ ]:
%pip install google-search-results

In [7]:
from langchain_openai.chat_models import ChatOpenAI
from langchain.agents import AgentType, initialize_agent, load_tools

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
tools = load_tools(["serpapi", "llm-math"], llm=llm, serpapi_api_key = serpapi_api_key)
agent = initialize_agent(
    tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=False
)

C:\Users\User\AppData\Local\Temp\ipykernel_12008\1682705739.py:6: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(


In [8]:
import asyncio

inputs = [
    "How many active satellites are orbiting Earth as of 2024?",
    "Who won the Nobel Prize in Physics in 2023?",
    "What is the tallest building in the world, and what is its height raised to the power of 0.27?",
    "How many episodes are there in the anime One Piece as of 2024?",
    "What is the distance between the Earth and Mars in miles?",
    "What is the inflation rate in the United States in 2024?",
    "How many gold medals did China win in the 2024 Olympics?",
    "What is the population of India multiplied by 0.0025?",
    "Who is the author of Dune and what year was it published?",
    "What is the melting point of iron in Kelvin?"
]

results = []


async def arun(agent, input_example):
    try:
        return await agent.ainvoke(input_example)
    except Exception as e:
        # The agent sometimes makes mistakes! These will be captured by the tracing.
        return e


for input_example in inputs:
    results.append(arun(agent, input_example))
results = await asyncio.gather(*results)

In [11]:
results

[{'input': 'How many active satellites are orbiting Earth as of 2024?',
  'output': 'As of 2024, there are more than 10,400 actively functioning satellites orbiting Earth.'},
 {'input': 'Who won the Nobel Prize in Physics in 2023?',
  'output': "Pierre Agostini, Ferenc Krausz, and Anne L'Huillier."},
 {'input': 'What is the tallest building in the world, and what is its height raised to the power of 0.27?',
  'output': '6.14 (rounded to two decimal places)'},
 {'input': 'How many episodes are there in the anime One Piece as of 2024?',
  'output': 'As of 2024, there are 1,122 episodes of the anime One Piece.'},
 {'input': 'What is the distance between the Earth and Mars in miles?',
  'output': 'The distance between Earth and Mars varies from about 37 million miles at closest approach to approximately 140 million miles on average, and can reach up to about 250 million miles at its farthest.'},
 {'input': 'What is the inflation rate in the United States in 2024?',
  'output': 'The inflati

In [9]:
from langchain.callbacks.tracers.langchain import wait_for_all_tracers

# Logs are submitted in a background thread to avoid blocking execution.
# For the sake of this tutorial, we want to make sure
# they've been submitted before moving on. This is also
# useful for serverless deployments.
wait_for_all_tracers()